In [2]:
import glob,os
import pandas as pd
import numpy as np
import requests
import datetime
from openai import OpenAI
import tiktoken
import time
import re
import pinecone

def read_sp500():
    df = pd.read_csv("../data/sp500.csv")
    return df
def read_nikkei_symbols():
    df = pd.read_csv("../data/nikkei_symbols.csv")
    df.columns=["symbol"]
    return df
sp500=read_sp500()["symbol"].tolist()
nikkei=read_nikkei_symbols()["symbol"].tolist()
tickers = sp500+nikkei

pd.set_option('display.max_columns', None)

def read_all_data():
    files = glob.glob("../data/all_*.csv")
    all_data={}
    for file_path in files:
        name=os.path.basename(file_path)[4:-4]
        print(name)
        # read
        df=pd.read_csv(file_path)
        # datetime
        if name in ["price_targets","grades","general_news"]:
            df["datetime"]=pd.to_datetime(df["publishedDate"].map(lambda s:s.replace('\n', '')),format='%Y-%m-%dT%H:%M:%S.000Z')
        elif name in ["stock_prices","articles","earnings","ratings","analyst_recommendations","analyst_estimates","social_sentiments","sector_performances","stock_grades","incomes","balance_sheets","cashflows","earnings_surprises","enterprise_values","key_metrics","ratios"]:
            df["datetime"]=pd.to_datetime(df["date"])
        elif name in ["stock_news"]:
            df["datetime"]=pd.to_datetime(df["publishedDate"])
        all_data[name]=df
    all_data["stock_prices"]=all_data["stock_prices"].loc[all_data["stock_prices"]["datetime"]>=pd.to_datetime("2000-01-01"),:]
    return all_data
all_data=read_all_data()

c:\Users\n1201023.STCN2\AppData\Local\anaconda3\envs\py10\lib\site-packages\pinecone\index.py:4: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from tqdm.autonotebook import tqdm


analyst_estimates
analyst_recommendations
articles
balance_sheets


C:\Users\n1201023.STCN2\AppData\Local\Temp\ipykernel_35084\3972868514.py:32: DtypeWarning: Columns (52,53) have mixed types. Specify dtype option on import or set low_memory=False.
  df=pd.read_csv(file_path)


cashflows


C:\Users\n1201023.STCN2\AppData\Local\Temp\ipykernel_35084\3972868514.py:32: DtypeWarning: Columns (38,39) have mixed types. Specify dtype option on import or set low_memory=False.
  df=pd.read_csv(file_path)


company_profiles
earnings
earnings_surprises
enterprise_values
general_news
grades
incomes


C:\Users\n1201023.STCN2\AppData\Local\Temp\ipykernel_35084\3972868514.py:32: DtypeWarning: Columns (36,37) have mixed types. Specify dtype option on import or set low_memory=False.
  df=pd.read_csv(file_path)


insider_trades
key_metrics
price_targets
ratings
ratios
sector_performances
social_sentiments
stock_grades
stock_news
stock_peers
stock_prices
symbols


In [32]:
start_predict_dt=pd.to_datetime("2023-05-01")
end_predict_dt=pd.to_datetime("2023-11-30")
max_general_news_delay=90 # 最大何日過去のニュースを利用するか
pinecone_api_key="bc02d80d-c2d5-4dce-b80f-5506511bc2bc"

In [12]:
def read_csv_with_ada_embedding(path):
    df = pd.read_csv(path)
    df['ada_embedding'] = df.ada_embedding.apply(eval).apply(np.array)
    return df
embedded_task1_queries=read_csv_with_ada_embedding('../data/embedded_task1_queries.csv')
embedded_general_news=read_csv_with_ada_embedding('../data/embedded_general_news.csv')
embedded_general_news["datetime"]=pd.to_datetime(embedded_general_news["datetime"])

# text-embedding-ada-002

In [1]:
client = OpenAI()
# embedding model parameters
embedding_model = "text-embedding-ada-002"
embedding_encoding = "cl100k_base"  # this the encoding for text-embedding-ada-002

In [206]:
df=all_data["general_news"]

df["embedding_text"]="Title: "+df["title"]+"; Content: "+df["text"]
df["embedding_text"]=df["embedding_text"].map(lambda x: x.replace("\n"," "))
df.dropna(subset=["embedding_text"],inplace=True)
encoding = tiktoken.get_encoding(embedding_encoding)
df["num_tokens"]=df["embedding_text"].map(lambda x: len(encoding.encode(x)))

use_news_site=['skynews','aljazeera','time','globalnews','cnn','buzzfeed']
df=df.loc[df["site"].isin(use_news_site),:]
df=df.loc[df["datetime"]>(start_predict_dt-datetime.timedelta(days=max_general_news_delay)),:]
df.drop_duplicates(subset=["embedding_text"],inplace=True)
df.reset_index(drop=True,inplace=True)

In [208]:
print(df.num_tokens.sum(),"tokens",df.num_tokens.sum()*0.0004/1000,"dollars")

1112940 tokens 0.44517600000000007 dollars


In [211]:
import time
embedding_model="text-embedding-ada-002"
embedding_texts=[]
token_sum=0
max_input=2048 # inputのlistの最大長は2048 https://github.com/openai/openai-python/issues/519
max_tmp=150000*2/3
responses=[]
for i,row in df.iterrows():
    token_sum+=row["num_tokens"]
    if (token_sum>=max_tmp) or (len(embedding_texts)>=max_input):
        print(i,"/",df.shape[0],len(embedding_texts),token_sum-row["num_tokens"])
        response=client.embeddings.create(input = embedding_texts, model=embedding_model)
        responses+=response.data
        token_sum=row["num_tokens"]
        embedding_texts=[row["embedding_text"]]
        time.sleep(65)
    else:
        embedding_texts+=[row["embedding_text"]]
    
responses+=client.embeddings.create(input = embedding_texts, model=embedding_model).data
df["ada_embedding"]=[r.embedding for r in responses]
df.to_csv('../data/embedded_general_news.csv', index=False)

2048 / 25797 2048 87399
4096 / 25797 2048 87173
6144 / 25797 2048 88035
8192 / 25797 2048 87956
10240 / 25797 2048 87551
12288 / 25797 2048 86650
14336 / 25797 2048 86792
16384 / 25797 2048 85770
18432 / 25797 2048 86801
20480 / 25797 2048 90768
22528 / 25797 2048 91955
24576 / 25797 2048 91883


In [ ]:
df = pd.read_csv('../data/embedded_general_news.csv')

## embedding task1 queries

In [382]:
df=pd.DataFrame(tickers,columns=["symbol"])
for i,row in df.iterrows():
    symbol=row["symbol"]
    profile=all_data["company_profiles"].loc[all_data["company_profiles"]["symbol"]==symbol,:].iloc[0,:]
    query=f'Wanted data: news related to the stock price of stock {symbol}; Description of stock {symbol}: {profile["description"]}'
    df.loc[i,"embedding_text"]=query.replace("\n"," ")
encoding = tiktoken.get_encoding(embedding_encoding)
df["num_tokens"]=df["embedding_text"].map(lambda x: len(encoding.encode(x)))
df.reset_index(drop=True,inplace=True)

In [ ]:
print(df.num_tokens.sum(),"tokens",df.num_tokens.sum()*0.0004/1000,"dollars")

In [386]:
import time
embedding_model="text-embedding-ada-002"
embedding_texts=[]
token_sum=0
max_input=2048 # inputのlistの最大長は2048 https://github.com/openai/openai-python/issues/519
max_tmp=150000*2/3
responses=[]
for i,row in df.iterrows():
    token_sum+=row["num_tokens"]
    if (token_sum>=max_tmp) or (len(embedding_texts)>=max_input):
        print(i,"/",df.shape[0],len(embedding_texts),token_sum-row["num_tokens"])
        response=client.embeddings.create(input = embedding_texts, model=embedding_model)
        responses+=response.data
        token_sum=row["num_tokens"]
        embedding_texts=[row["embedding_text"]]
        time.sleep(65)
    else:
        embedding_texts+=[row["embedding_text"]]

responses+=client.embeddings.create(input = embedding_texts, model=embedding_model).data
df["ada_embedding"]=[r.embedding for r in responses]
df.to_csv('../data/embedded_task1_queries.csv', index=False)
embedded_task1_queries=df

353 / 728 353 99969
696 / 728 343 99812


In [ ]:
df = pd.read_csv('../data/embedded_task1_queries.csv')
df['ada_embedding'] = df.ada_embedding.apply(eval).apply(np.array)

# insert data into Pinecone

In [658]:
%pip install pinecone-client
import pinecone
pinecone.init(api_key=pinecone_api_key, environment="gcp-starter")

Note: you may need to restart the kernel to use updated packages.


In [219]:
index_name = "sotsuken"
pinecone.create_index(index_name, dimension=1536, metric="cosine")
pinecone.describe_index(index_name)
index = pinecone.Index(index_name)

IndexDescription(name='sotsuken', metric='cosine', replicas=1, dimension=1536.0, shards=1, pods=1, pod_type='starter', status={'ready': True, 'state': 'Ready'}, metadata_config=None, source_collection='')

In [276]:
df=embedded_general_news
# meke upserts list
upserts = [None]*df.shape[0]
for i, row in df.iterrows():
    upserts[i]=(str(i), row["ada_embedding"],{"date":int(row["datetime"].strftime('%Y%m%d')),"site":row["site"]})
# upsert
upsert_step=100
for i in range(0,len(upserts),upsert_step):
    index.upsert(upserts[i:i+upsert_step])
    print(i,"/",len(upserts))

0 / 25797
100 / 25797
200 / 25797
300 / 25797
400 / 25797
500 / 25797
600 / 25797
700 / 25797
800 / 25797
900 / 25797
1000 / 25797
1100 / 25797
1200 / 25797
1300 / 25797
1400 / 25797
1500 / 25797
1600 / 25797
1700 / 25797
1800 / 25797
1900 / 25797
2000 / 25797
2100 / 25797
2200 / 25797
2300 / 25797
2400 / 25797
2500 / 25797
2600 / 25797
2700 / 25797
2800 / 25797
2900 / 25797
3000 / 25797
3100 / 25797
3200 / 25797
3300 / 25797
3400 / 25797
3500 / 25797
3600 / 25797
3700 / 25797
3800 / 25797
3900 / 25797
4000 / 25797
4100 / 25797
4200 / 25797
4300 / 25797
4400 / 25797
4500 / 25797
4600 / 25797
4700 / 25797
4800 / 25797
4900 / 25797
5000 / 25797
5100 / 25797
5200 / 25797
5300 / 25797
5400 / 25797
5500 / 25797
5600 / 25797
5700 / 25797
5800 / 25797
5900 / 25797
6000 / 25797
6100 / 25797
6200 / 25797
6300 / 25797
6400 / 25797
6500 / 25797
6600 / 25797
6700 / 25797
6800 / 25797
6900 / 25797
7000 / 25797
7100 / 25797
7200 / 25797
7300 / 25797
7400 / 25797
7500 / 25797
7600 / 25797
7700 / 2579

# task

## task1

In [32]:
import pinecone
pinecone.init(api_key=pinecone_api_key, environment="gcp-starter")
index_name = "sotsuken"
index = pinecone.Index(index_name)

def excute_task1(symbol,embedded_task1_queries,embedded_general_news,end_dt,delay,top_k=3):
    # get query embedding
    query_embedding=embedded_task1_queries.loc[embedded_task1_queries["symbol"]==symbol,"ada_embedding"].iloc[0]
    # search
    delay=datetime.timedelta(days=delay)
    start_date_int=int((end_dt-delay).strftime('%Y%m%d'))
    end_date_int=int(end_dt.strftime('%Y%m%d')) 
    result=index.query(
        vector=query_embedding.tolist(),
        filter={
            "$and":[{"date": {"$lte": end_date_int}},{"date": {"$gte": start_date_int}}]
        },
        top_k=top_k,
        include_metadata=True
    )
    # make result df
    ids=[int(d["id"]) for d in result["matches"]]
    scores=[d["score"] for d in result["matches"]]
    sites=[d["metadata"]["site"] for d in result["matches"]]
    result_df=pd.DataFrame({"id":ids,"score":scores,"site":sites})
    result_df=pd.merge(result_df,embedded_general_news,left_on="id",right_index=True).sort_values("score",ascending=False)
    result_df.rename(columns={"site_x":"site"},inplace=True)
    result_df.drop(columns=["site_y"],inplace=True)
    return result_df

## task2

In [34]:
import re
from openai import OpenAI
import tiktoken
client = OpenAI()
def excute_llm(messages,model="gpt-3.5-turbo",temperature=0.5,max_tokens=1000):
    response = client.chat.completions.create(
      model=model,
      temperature=temperature,
      max_tokens=max_tokens,
      messages=messages
    )
    return response

def make_task2_query(symbol,all_data,news_df):
    stock_info=all_data["company_profiles"].loc[all_data["company_profiles"]["symbol"]==symbol,"description"].iloc[0]
    news_query=""
    for i,row in news_df.iterrows():
        #task2_result=row["task2_result"]
        #news_query+=f"News {i}: {row["news_info"]}; Result of impact analysis on stock {symbol}: {task2_result} \n"
        news_query+=f"News {i+1}: {row['title']}\n"
    roles=["system","user","assistant","user"]
    contents=[f"You are Frederick, Sam and Chloe, excellent traders. You have been given the task of analyzing how the price of a stock {symbol} will fluctuate as a result of several news events.",
              f"You will be asked to analyze, step by step, how the price of stock A will fluctuate as a result of the multiple news items you are about to pass on, based on the information in the stock {symbol}, your knowledge as traders, and investor psychology. Do you understand?",
              f"Yes, I understand. We, Frederick, Sam and Chloe, traders, analyze how the stock {symbol}'s price fluctuates as a result of multiple news stories.",
              f"Excellent! Let's begin. \nStock {symbol} information: {stock_info} \n{news_query} Options: (1) rise significantly, (2) rise, (3) no change, (4) fall, (5) fall significantly \nYou three excellent traders, Frederick, Sam, and Chloe, will analyze all of this information according to the following steps and produce a result from your choices. First, write down your thoughts for one step separately for each of the three of you, and then share them with the three of you. Once that is done, pros price will fluctuate as a ceed to the next step. If you notice any mistakes, correct them. Finally, if the three of you agree on the same option, end the discussion. Now, analyze step by step how the stock {symbol}'result of multiple news stories, based on the above information, your knowledge as traders, and investor psychology, and then discuss and draw a conclusion. The final answer that you all agreed upon should be in the form \"Our Final Answer: (1) rise significantly\". Be sure to limit your answer to one option."]
    messages=[{"role":role,"content":content} for role,content in zip(roles,contents)]
    return messages

def find_task2_final_decision(llm_output):
    fdidx=llm_output.rfind("Our Final Answer")
    if fdidx is not None:
        m=re.search(r"\([0-9]\)",llm_output[fdidx:fdidx+30])
        if m is not None:
            return int(m.group()[1])
        else:
            return None
    else:
        return None
    
def retry_to_find_task2_final_decision(llm_output,max_retry=3):
    roles=["assistant","user"]
    contents=[llm_output,
              f"The final answer that you all agreed upon should be in the form \"Our Final Answer: (number) choice\". The choices are (1) rise significantly, (2) rise, (3) no change, (4) fall, and (5) fall significantly."]
    messages=[{"role":role,"content":content} for role,content in zip(roles,contents)]
    i=0
    while True:
        response=excute_llm(messages,temperature=0.8)
        retry_output=response.choices[0].message.content
        fd=find_task2_final_decision(retry_output)
        if fd is not None:
            return (fd,retry_output)
        else:
            if i>=max_retry:
                return None
            print("retry",i)
            i+=1
            
def check_query_tokens(messages,max_tokens=4097,embedding_encoding="cl100k_base"):
    encoding = tiktoken.get_encoding(embedding_encoding)
    tokens=[len(encoding.encode(m["content"])) for m in messages]
    if sum(tokens)>max_tokens:
        print("too many tokens",sum(tokens))
        return False
    else:
        return True

def excute_task2(messages):
    response=excute_llm(messages,max_tokens=3000)
    llm_output=response.choices[0].message.content
    input_tokens=response.usage.prompt_tokens
    output_tokens=response.usage.completion_tokens
    retry_output=None
    # find final decision
    task2_result=find_task2_final_decision(llm_output)
    if task2_result is None:
        task2_result,retry_output=retry_to_find_task2_final_decision(llm_output)
    cost=(0.001*input_tokens/1000)+(0.002*output_tokens/1000)
    #print("cost:",cost,"dollars")
    return llm_output,task2_result,cost,retry_output

## task2 additional

In [476]:
def make_task2_additional_query(symbol,all_data,news_df):
    stock_info=all_data["company_profiles"].loc[all_data["company_profiles"]["symbol"]==symbol,"description"].iloc[0]
    news_query=""
    for i,row in news_df.iterrows():
        #task2_additional_result=row["task2_additional_result"]
        #news_query+=f"News {i}: {row["news_info"]}; Result of impact analysis on stock {symbol}: {task2_additional_result} \n"
        news_query+=f"News {i+1}: {row['title']}\n"
    roles=["system","user","assistant","user"]
    contents=[f"You are Frederick, Sam and Chloe, excellent traders. You have been given the task of analyzing how the price of a stock {symbol} will fluctuate.",
              f"You will be asked to analyze, step by step, how the price of stock A will fluctuate as a result of the multiple news items you are about to pass on, based on the information in the stock {symbol}, your knowledge as traders, and investor psychology. Do you understand?",
              f"Yes, I understand. We, Frederick, Sam and Chloe, traders, analyze how the stock {symbol}'s price fluctuates as a result of multiple news stories.",
              f"Excellent! Let's begin. \nStock {symbol} information: {stock_info} \n{news_query} Options: (1) Rise, (2) Fall, (3) Unknown\nYou three excellent traders, Frederick, Sam, and Chloe, will analyze all of this information according to the following steps and produce a result from your choices. First, write down your thoughts for one step separately for each of the three of you, and then share them with the three of you. Once that is done, pros price will fluctuate as a ceed to the next step. If you notice any mistakes, correct them. Finally, if the three of you agree on the same option, end the discussion. Now, analyze step by step how the stock {symbol}'result of multiple news stories, based on the above information, your knowledge as traders, and investor psychology, and then discuss and draw a conclusion. The final answer that you all agreed upon should be in the form \"Our Final Answer: (1) Rise\". Be sure to limit your answer to one option. If you cannot agree on a final answer, choose (3) Unknown."]
    messages=[{"role":role,"content":content} for role,content in zip(roles,contents)]
    return messages

def find_task2_additional_final_decision(llm_output):
    fdidx=llm_output.rfind("Our Final Answer")
    if fdidx is not None:
        m=re.search(r"\([0-9]\)",llm_output[fdidx:fdidx+30])
        if m is not None:
            return int(m.group()[1])
        else:
            return None
    else:
        return None
    
def retry_to_find_task2_additional_final_decision(llm_output,max_retry=3):
    roles=["assistant","user"]
    contents=[llm_output,
              f"The final answer that you all agreed upon should be in the form \"Our Final Answer: (number) choice\". The choices are  (1) Rise, (2) Fall, (3) Unknown. Be sure to limit your answer to one option.If you cannot agree on a final answer, choose (3) Unknown."]
    messages=[{"role":role,"content":content} for role,content in zip(roles,contents)]
    i=0
    while True:
        response=excute_llm(messages,temperature=0.8)
        option_output=response.choices[0].message.content
        fd=find_task2_additional_final_decision(option_output)
        if fd is not None:
            return fd
        else:
            if i>=max_retry:
                return None
            print("retry",i)
            i+=1
            
def check_query_tokens(messages,max_tokens=4097,embedding_encoding="cl100k_base"):
    encoding = tiktoken.get_encoding(embedding_encoding)
    tokens=[len(encoding.encode(m["content"])) for m in messages]
    if sum(tokens)>max_tokens:
        print("too many tokens",sum(tokens))
        return False
    else:
        return True

def excute_task2_additional(messages):
    response=excute_llm(messages,max_tokens=3000)
    llm_output=response.choices[0].message.content
    input_tokens=response.usage.prompt_tokens
    output_tokens=response.usage.completion_tokens
    # find final decision
    task2_additional_result=find_task2_additional_final_decision(llm_output)
    if task2_additional_result is None:
        task2_additional_result=retry_to_find_task2_additional_final_decision(llm_output)
    cost=(0.001*input_tokens/1000)+(0.002*output_tokens/1000)
    return llm_output,task2_additional_result,cost

## concat stock_news and embedded_general_news

In [33]:
# pick only use stock_news
dow30=pd.read_csv("../data/dow30.csv")["symbol"].tolist()
stock_news=all_data["stock_news"].loc[(all_data["stock_news"]["datetime"]>=start_predict_dt)&(all_data["stock_news"]["datetime"]<=end_predict_dt),:]
stock_news=stock_news.loc[stock_news["symbol"].isin(dow30),:]
use_news_site=["Reuters"]
stock_news=stock_news.loc[stock_news["site"].isin(use_news_site),:]
stock_news=stock_news.drop_duplicates(subset="title",keep="last").reset_index(drop=True)

In [35]:
# task2で使うニュースをdfに格納　
def get_news_df(symbol,stock_news,general_news,end_dt,delay):
    delay=datetime.timedelta(days=delay)
    start_dt=end_dt-delay
    news_df=stock_news.loc[stock_news["symbol"]==symbol,["datetime","title","text","site"]]
    news_df=news_df.loc[news_df["datetime"]>=start_dt,:]
    news_df=news_df.loc[news_df["datetime"]<=end_dt,:]
    tmp=general_news.loc[:,["datetime","title","text","site"]]
    news_df=pd.concat([news_df,tmp],axis=0)
    news_df["news_info"]="Title: "+news_df["title"]+"; Content: "+news_df["text"]
    news_df.drop_duplicates(subset="news_info",keep="last",inplace=True)
    news_df.reset_index(drop=True,inplace=True)
    return news_df

# excute tasks

In [36]:
task2_df=stock_news

In [ ]:
delay=2
top_k=2
llm_outputs=[]
task2_results=[]
costs=[]
messages_list=[]
retry_outputs=[]

In [ ]:
# 制限に引っかかったらstart_idxを変えて再開
start_idx=0
save_name=f"../data/task2_{datetime.datetime.now().strftime('%Y%m%d%H%M%S')}.csv"
for i in range(start_idx,task2_df.shape[0]):
    print("start:",i,"/",task2_df.shape[0]-1)
    row=task2_df.loc[i,:]
    symbol=row["symbol"]
    end_dt=row["datetime"]
    # pick 
    general_news=excute_task1(symbol,embedded_task1_queries,embedded_general_news,end_dt,delay=delay,top_k=top_k)
    news_df=get_news_df(symbol,stock_news,general_news,end_dt,delay=delay)
    #print(f"task1: news_df.shape",news_df.shape)
    messages=make_task2_query(symbol,all_data,news_df)
    llm_output,task2_result,cost,retry_output=excute_task2(messages)
    task2_df.loc[i,"llm_output"]=llm_output
    task2_df.loc[i,"task2_result"]=task2_result
    task2_df.loc[i,"cost"]=cost
    task2_df.loc[:,"messages"].loc[i]=messages
    task2_df.loc[i,"retry_output"]=retry_output
    task2_df.to_csv(save_name,index=False)
    print("end:",i,"cost:",cost,"dollars")

# excute additional tasks

In [473]:
task2_additional_df=task2_df.loc[task2_df["symbol"]=="CVX",:].reset_index(drop=True)

In [472]:
delay=2
top_k=2
llm_outputs=[]
task2_additional_results=[]
costs=[]
messages_list=[]

In [ ]:
# 制限に引っかかったらstart_idxを変えて再開
start_idx=0
for i in range(start_idx,task2_additional_df.shape[0]):
    row=task2_additional_df.loc[i,:]
    symbol=row["symbol"]
    end_date=row["datetime"]
    general_news=excute_task1(symbol,embedded_task1_queries,embedded_general_news,end_date,delay=2,top_k=2)
    news_df=get_news_df(symbol,stock_news,general_news,end_date,delay=2)
    print("start:",i,"/",task2_additional_df.shape[0]-1,"news_df.shape",news_df.shape)
    messages=make_task2_additional_query(symbol,all_data,news_df)
    llm_output,task2_additional_result,cost=excute_task2_additional(messages)
    task2_additional_df.loc[i,"llm_output"]=llm_output
    task2_additional_df.loc[i,"task2_additional_result"]=task2_additional_result
    task2_additional_df.loc[i,"cost"]=cost
    task2_additional_df.loc[:,"messages"].loc[i]=messages
    task2_additional_df.to_csv("../data/task2_additional.csv",index=False)
    print("end:",i,"cost:",cost,"dollars")

# change rate (not use)

In [619]:
import math
def add_u_and_s(symbol,all_data,n):
    df=all_data["stock_prices"].loc[all_data["stock_prices"]["symbol"]==symbol,:]
    df.sort_values("datetime",ascending=False,inplace=True)
    df.reset_index(drop=True,inplace=True)
    s_i=df.iloc[0:-1,:].loc[:,"adjClose"].reset_index(drop=True)
    s_i1=df.iloc[1:,:].loc[:,"adjClose"].reset_index(drop=True)
    u=(s_i-s_i1)/s_i1
    df.loc[:,"u"]=u
    s=[]
    for i in range(df.shape[0]-n):
        s+=[math.sqrt(((1/(n-1))*(sum([(u[k]-u[i:i+n].mean())**2 for k in range(i,i+n)]))))]
    df["s"]=pd.DataFrame(s)
    df["u/s"]=df["u"]/df["s"]
    return df.sort_values("datetime",ascending=True,ignore_index=True)
def add_u_and_s_symbols(symbols,all_data,n):
    dfs=[]
    for symbol in symbols:
        df=add_u_and_s(symbol,all_data,n)
        dfs.append(df)
    return pd.concat(dfs,axis=0)

# evaluation

In [49]:
price_df=all_data["stock_prices"].loc[all_data["stock_prices"]["datetime"]>=pd.to_datetime("2023-01-01"),:]

In [6]:
def read_task2_df(path="../data/task2.csv"):
    df=pd.read_csv(path)
    df["datetime"]=pd.to_datetime(df["datetime"])
    for i,row in df.iterrows():
        df.loc[:,"messages"].loc[i]=eval(row["messages"])
    return df

In [53]:
task2_df=read_task2_df()
task2_additional_df=read_task2_df("../data/task2_additional.csv")

C:\Users\n1201023.STCN2\AppData\Local\Temp\ipykernel_35084\3344682907.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df.loc[:,"messages"].loc[i]=eval(row["messages"])
C:\Users\n1201023.STCN2\AppData\Local\Temp\ipykernel_35084\3344682907.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df.loc[:,"messages"].loc[i]=eval(row["messages"])
C:\Users\n1201023.STCN2\AppData\Local\Temp\ipykernel_35084\3344682907.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#return

In [50]:
def calc_strategy_acc(price_df,task2_df,strategy={"buy":[1,2],"sell":[4,5],"hold":[3]},type="now",delay=1,result_col="task2_result"):
    df=pd.DataFrame()
    for _,row in task2_df.iterrows():
        p=price_df.loc[price_df["symbol"]==row["symbol"],:].sort_values("datetime",ascending=True,ignore_index=True)
        date=pd.to_datetime(row["datetime"].date())
        i=p.loc[p["datetime"]>=date,:].index[0]
        if type=="pre":
            pre=p.loc[i-1,:]
        now=p.loc[i,:]
        t2_r=row[result_col]
        if (t2_r in strategy["buy"]) or (t2_r in strategy["sell"]) or (t2_r in strategy["hold"]):
            next=p.loc[i+delay,:]
            tmp=pd.DataFrame([row])
            if type=="pre":
                tmp["next_price"]=next["adjClose"]
                tmp["pre_price"]=pre["adjClose"]
                tmp["diff"]=next["adjClose"]-pre["adjClose"]
                tmp["change"]=(next["adjClose"]-pre["adjClose"])/pre["adjClose"]
                tmp["acc"]=1 if ((t2_r in strategy["buy"]) and next["adjClose"]>pre["adjClose"]) or ((t2_r in strategy["sell"]) and next["adjClose"]<pre["adjClose"]) else 0
            elif type=="now":
                tmp["now_price"]=now["adjClose"]
                tmp["next_price"]=next["adjClose"]
                tmp["diff"]=next["adjClose"]-now["adjClose"]
                tmp["change"]=(next["adjClose"]-now["adjClose"])/now["adjClose"]
                tmp["acc"]=1 if ((t2_r in strategy["buy"]) and next["adjClose"]>now["adjClose"]) or ((t2_r in strategy["sell"]) and next["adjClose"]<now["adjClose"]) else 0
            df=pd.concat([df,tmp],axis=0)
    return df

def print_task2_evaluation(df,strategy={"buy":[1,2],"sell":[4,5],"hold":[3]},result_col="task2_result"):
    print(df[result_col].shape[0])
    print(df[result_col].value_counts().sort_index())
    print("true",df.loc[df["acc"]==1,result_col].value_counts().sort_index())
    print("false",df.loc[df["acc"]==0,result_col].value_counts().sort_index())
    TP=df.loc[(df["acc"]==1)&(df[result_col].isin(strategy["buy"])),:].shape[0]
    FN=df.loc[(df["acc"]==0)&(df[result_col].isin(strategy["sell"])),:].shape[0]
    FP=df.loc[(df["acc"]==0)&(df[result_col].isin(strategy["buy"])),:].shape[0]
    TN=df.loc[(df["acc"]==1)&(df[result_col].isin(strategy["sell"])),:].shape[0]
    #print("TP",TP,"FN",FN,"FP",FP,"TN",TN)
    print("actual\pred","fall","fall")
    print("rise",TP,FN)
    print("fall",FP,TN)
    print("accuracy",(TP+TN)/(TP+TN+FP+FN))
    print("precision",TP/(TP+FP))
    print("recall",TP/(TP+FN))
    print("F1",2*TP/(2*TP+FP+FN))

## task2_acc

In [58]:
strategy={"buy":[1,2],"sell":[4,5],"hold":[3]}
result_col="task2_result"
task2_acc=calc_strategy_acc(price_df,task2_df,strategy=strategy,type="now",delay=1,result_col=result_col)
print_task2_evaluation(task2_acc,strategy=strategy,result_col=result_col)
task2_acc.to_csv("../data/task2_acc_now.csv",index=False)

525
task2_result
1.0     23
2.0    195
3.0     65
4.0    236
5.0      6
Name: count, dtype: int64
true task2_result
1.0     12
2.0     96
4.0    105
5.0      4
Name: count, dtype: int64
false task2_result
1.0     11
2.0     99
3.0     65
4.0    131
5.0      2
Name: count, dtype: int64
actual\pred fall fall
rise 108 133
fall 110 109
accuracy 0.4717391304347826
precision 0.4954128440366973
recall 0.44813278008298757
F1 0.47058823529411764


In [59]:
strategy={"buy":[1,2],"sell":[4,5],"hold":[3]}
result_col="task2_result"
task2_acc=calc_strategy_acc(price_df,task2_df,strategy=strategy,type="pre",delay=1,result_col=result_col)
print_task2_evaluation(task2_acc,strategy=strategy,result_col=result_col)
task2_acc.to_csv("../data/task2_acc_pre.csv",index=False)

525
task2_result
1.0     23
2.0    195
3.0     65
4.0    236
5.0      6
Name: count, dtype: int64
true task2_result
1.0     13
2.0     97
4.0    113
5.0      6
Name: count, dtype: int64
false task2_result
1.0     10
2.0     98
3.0     65
4.0    123
Name: count, dtype: int64
actual\pred fall fall
rise 110 123
fall 108 119
accuracy 0.49782608695652175
precision 0.5045871559633027
recall 0.4721030042918455
F1 0.4878048780487805


## task2_additional_acc

In [60]:
strategy={"buy":[1],"sell":[2],"hold":[3]}
result_col="task2_additional_result"
task2_additional_acc=calc_strategy_acc(price_df,task2_additional_df,strategy=strategy,type="now",delay=1,result_col=result_col)
print_task2_evaluation(task2_additional_acc,strategy=strategy,result_col=result_col)
task2_acc.to_csv("../data/task2_additional_acc_now.csv",index=False)

94
task2_additional_result
1.0    21
2.0    25
3.0    48
Name: count, dtype: int64
true task2_additional_result
1.0     9
2.0    10
Name: count, dtype: int64
false task2_additional_result
1.0    12
2.0    15
3.0    48
Name: count, dtype: int64
actual\pred fall fall
rise 9 15
fall 12 10
accuracy 0.41304347826086957
precision 0.42857142857142855
recall 0.375
F1 0.4


In [61]:
strategy={"buy":[1],"sell":[2],"hold":[3]}
result_col="task2_additional_result"
task2_additional_acc=calc_strategy_acc(price_df,task2_additional_df,strategy=strategy,type="pre",delay=1,result_col=result_col)
print_task2_evaluation(task2_additional_acc,strategy=strategy,result_col=result_col)
task2_acc.to_csv("../data/task2_additional_acc_pre.csv",index=False)

94
task2_additional_result
1.0    21
2.0    25
3.0    48
Name: count, dtype: int64
true task2_additional_result
1.0     6
2.0    10
Name: count, dtype: int64
false task2_additional_result
1.0    15
2.0    15
3.0    48
Name: count, dtype: int64
actual\pred fall fall
rise 6 15
fall 15 10
accuracy 0.34782608695652173
precision 0.2857142857142857
recall 0.2857142857142857
F1 0.2857142857142857


## buy and hold

In [62]:
def buy_and_hold_symbol(symbol,price_df,task2_df,strategy,result_col):
    p=price_df.loc[price_df["symbol"]==symbol,:].sort_values("datetime",ascending=True,ignore_index=True)
    t2=task2_df.loc[task2_df["symbol"]==symbol,:].sort_values("datetime",ascending=True,ignore_index=True)
    profit=0
    invest=0
    pre_strategy="sell"
    buy_price=0
    for _,row in t2.iterrows():
        i=p.loc[p["datetime"]<=row["datetime"],:].index[-1]
        now=p.loc[i,:]
        pre=p.loc[i-1,:]
        t2_r=row[result_col]
        r_strategy="buy" if t2_r in strategy["buy"] else "sell" if t2_r in strategy["sell"] else "hold"
        if (r_strategy!=pre_strategy) and (t2_r in strategy["buy"]+strategy["sell"]+strategy["hold"]):
            if t2_r in strategy["buy"]:
                buy_price=now["adjClose"]
                pre_strategy="buy"
                print("buy",now["datetime"],now["adjClose"])
            elif t2_r in strategy["sell"]:
                profit+=now["adjClose"]-buy_price
                invest+=buy_price
                pre_strategy="sell"
                print("sell",now["datetime"],now["adjClose"],buy_price)
    if pre_strategy=="buy":
        profit+=now["adjClose"]-buy_price
        invest+=buy_price
        print("sell",now["datetime"],now["adjClose"],buy_price)
    if invest==0:
        print("profit:",profit,"invest:",invest,"profit/invest:",0)
        return profit,invest,0
    else:
        print("profit:",profit,"invest:",invest,"profit/invest:",profit/invest)
        return profit,invest,profit/invest

def buy_and_hold_symbols(symbols,price_df,task2_df,strategy,result_col):
    profits=[]
    invests=[]
    profit_invests=[]
    symbol_list=[]
    for symbol in symbols:
        print(symbol)
        if task2_df.loc[task2_df["symbol"]==symbol,:].shape[0]==0:
            continue
        profit,invest,profit_invest=buy_and_hold_symbol(symbol,price_df,task2_df,strategy,result_col)
        symbol_list.append(symbol)
        profits.append(profit)
        invests.append(invest)
        profit_invests.append(profit_invest)
    df=pd.DataFrame({"symbol":symbol_list,"profit":profits,"invest":invests,"profit_invest":profit_invests})
    return df

In [63]:
strategy={"buy":[1,2],"sell":[4,5],"hold":[3]}
result_col="task2_result"
bh_df=buy_and_hold_symbols(dow30,price_df,task2_df,strategy,result_col)
bh_df.to_csv("../data/buy_and_hold.csv",index=False)

AMGN
buy 2023-05-18 00:00:00 220.69
sell 2023-06-27 00:00:00 219.1 220.69
buy 2023-08-25 00:00:00 254.38
sell 2023-09-05 00:00:00 252.03 254.38
buy 2023-09-14 00:00:00 260.25
sell 2023-10-03 00:00:00 258.97 260.25
buy 2023-10-31 00:00:00 253.71
sell 2023-11-09 00:00:00 262.0 253.71
profit: 3.0700000000000216 invest: 989.03 profit/invest: 0.0031040514443444805
CRM
HON
buy 2023-07-27 00:00:00 194.2
sell 2023-10-26 00:00:00 175.03 194.2
profit: -19.169999999999987 invest: 194.2 profit/invest: -0.09871266735324402
GS
buy 2023-06-01 00:00:00 311.23
sell 2023-06-15 00:00:00 334.19 311.23
buy 2023-06-26 00:00:00 307.25
sell 2023-06-26 00:00:00 307.25 307.25
buy 2023-06-29 00:00:00 317.81
sell 2023-07-05 00:00:00 314.82 317.81
buy 2023-09-07 00:00:00 319.34
sell 2023-09-13 00:00:00 332.25 319.34
buy 2023-11-13 00:00:00 324.25
sell 2023-11-14 00:00:00 335.96 324.25
profit: 44.589999999999975 invest: 1579.8799999999999 profit/invest: 0.028223662556649858
WBA
buy 2023-05-26 00:00:00 28.8
sell 202

In [65]:
bh_df["profit"].sum()/bh_df["invest"].sum()

0.002708952477746155

In [66]:
strategy={"buy":[1],"sell":[2],"hold":[3]}
result_col="task2_additional_result"
bh_df=buy_and_hold_symbols(dow30,price_df,task2_additional_df,strategy,result_col)
bh_df.to_csv("../data/buy_and_hold_additional.csv",index=False)

AMGN
CRM
HON
GS
WBA
AAPL
NKE
V
UNH
CSCO
TRV
CVX
buy 2023-05-22 00:00:00 149.42
sell 2023-05-25 00:00:00 151.5 149.42
buy 2023-07-25 00:00:00 159.34
sell 2023-08-10 00:00:00 157.65 159.34
buy 2023-08-22 00:00:00 157.39
sell 2023-08-24 00:00:00 156.3 157.39
buy 2023-08-29 00:00:00 158.3
sell 2023-08-30 00:00:00 158.52 158.3
buy 2023-09-21 00:00:00 163.48
sell 2023-10-05 00:00:00 162.2 163.48
buy 2023-10-09 00:00:00 164.99
sell 2023-10-27 00:00:00 142.85 164.99
buy 2023-10-30 00:00:00 144.57
sell 2023-11-02 00:00:00 147.22 144.57
buy 2023-11-06 00:00:00 145.48
sell 2023-11-28 00:00:00 145.51 145.48
profit: -21.21999999999997 invest: 1242.97 profit/invest: -0.017072013001118266
VZ
HD
INTC
MSFT
JNJ
WMT
CAT
DIS
DOW
JPM
BA
KO
MCD
AXP
IBM
MRK
MMM
PG


In [68]:
bh_df

,symbol,profit,invest,profit_invest
0,CVX,-21.22,1242.97,-0.017072


## my_strategy

In [69]:
symbol="AAPL"
strategy={"buy":[1,2],"sell":[4,5],"hold":[3]}
result_col="task2_result"
def my_strategy_symbol(symbol,price_df,task2_df,strategy,result_col):
    p=price_df.loc[price_df["symbol"]==symbol,:].sort_values("datetime",ascending=True,ignore_index=True)
    t2=task2_df.loc[task2_df["symbol"]==symbol,:].sort_values("datetime",ascending=True,ignore_index=True)
    profit=0
    invest=0
    pre_strategy="sell"
    buy_price=0
    for t2_i,row in t2.iterrows():
        i=p.loc[p["datetime"]<=row["datetime"],:].index[-1]
        now=p.loc[i,:]
        pre=p.loc[i-1,:]
        t2_r=row[result_col]
        r_strategy="buy" if t2_r in strategy["buy"] else "sell" if t2_r in strategy["sell"] else "hold"
        if (r_strategy!=pre_strategy) and (t2_r in strategy["buy"]+strategy["sell"]+strategy["hold"]):
            if t2_r in strategy["buy"]:
                buy_price=now["adjClose"]
                pre_strategy="buy"
                print("buy",row["datetime"],now["adjClose"])
                j=i+1
                while True:
                    if (t2_i+1>t2.shape[0]-1) or (t2.loc[t2_i+1,"datetime"]>p.loc[j,"datetime"]):
                        break
                    print(t2.loc[t2_i+1,"datetime"],p.loc[j,"datetime"])
                    pre_price=p.loc[j,"adjClose"]
                    j+=1
                    if pre_price>p.loc[j,"adjClose"]:
                        profit+=p.loc[j,"adjClose"]-buy_price
                        invest+=buy_price
                        pre_strategy="sell"
                        print("buy_sell",p.loc[j,"datetime"],p.loc[j,"adjClose"],buy_price)
                        break
            elif t2_r in strategy["sell"]:
                profit+=now["adjClose"]-buy_price
                invest+=buy_price
                pre_strategy="sell"
                print("sell",now["datetime"],now["adjClose"],buy_price)
    if pre_strategy=="buy":
        profit+=now["adjClose"]-buy_price
        invest+=buy_price
        print("sell",now["datetime"],now["adjClose"],buy_price)
    if invest==0:
        print("profit:",profit,"invest:",invest,"profit/invest:",0)
        return profit,invest,0
    else:
        print("profit:",profit,"invest:",invest,"profit/invest:",profit/invest)
        return profit,invest,profit/invest
def my_strategy_symbols(symbols,price_df,task2_df,strategy,result_col):
    profits=[]
    invests=[]
    profit_invests=[]
    symbol_list=[]
    for symbol in symbols:
        print(symbol)
        if task2_df.loc[task2_df["symbol"]==symbol,:].shape[0]==0:
            continue
        profit,invest,profit_invest=my_strategy_symbol(symbol,price_df,task2_df,strategy,result_col)
        symbol_list.append(symbol)
        profits.append(profit)
        invests.append(invest)
        profit_invests.append(profit_invest)
    df=pd.DataFrame({"symbol":symbol_list,"profit":profits,"invest":invests,"profit_invest":profit_invests})
    return df

In [70]:
strategy={"buy":[1,2],"sell":[4,5],"hold":[3]}
result_col="task2_result"
df=my_strategy_symbols(dow30,price_df,task2_df,strategy,result_col)
df.to_csv("../data/my_strategy.csv",index=False)

AMGN
buy 2023-05-18 06:04:06 220.69
2023-05-18 10:13:25 2023-05-19 00:00:00
2023-05-18 10:13:25 2023-05-22 00:00:00
2023-05-18 10:13:25 2023-05-23 00:00:00
buy_sell 2023-05-24 00:00:00 217.83 220.69
buy 2023-05-18 10:13:25 220.69
sell 2023-06-27 00:00:00 219.1 220.69
buy 2023-08-26 11:43:30 254.38
sell 2023-09-05 00:00:00 252.03 254.38
buy 2023-09-14 08:41:01 260.25
sell 2023-10-03 00:00:00 258.97 260.25
buy 2023-10-31 07:10:31 253.71
2023-10-31 18:32:13 2023-11-01 00:00:00
2023-10-31 18:32:13 2023-11-02 00:00:00
2023-10-31 18:32:13 2023-11-03 00:00:00
2023-10-31 18:32:13 2023-11-06 00:00:00
buy_sell 2023-11-07 00:00:00 268.99 253.71
buy 2023-10-31 18:32:13 253.71
sell 2023-11-09 00:00:00 262.0 253.71
profit: 15.490000000000038 invest: 1463.43 profit/invest: 0.010584722193750324
CRM
HON
buy 2023-07-27 07:50:37 194.2
sell 2023-10-26 00:00:00 175.03 194.2
profit: -19.169999999999987 invest: 194.2 profit/invest: -0.09871266735324402
GS
buy 2023-06-01 10:57:00 311.23
sell 2023-06-15 00:00:

In [71]:
df["profit"].sum()/df["invest"].sum()

0.004161032049270357

In [72]:
strategy={"buy":[1],"sell":[2],"hold":[3]}
result_col="task2_additional_result"
df=my_strategy_symbols(dow30,price_df,task2_additional_df,strategy,result_col)
df.to_csv("../data/my_strategy_additional.csv",index=False)

AMGN
CRM
HON
GS
WBA
AAPL
NKE
V
UNH
CSCO
TRV
CVX
buy 2023-05-22 08:06:32 149.42
sell 2023-05-25 00:00:00 151.5 149.42
buy 2023-07-25 09:36:07 159.34
sell 2023-08-10 00:00:00 157.65 159.34
buy 2023-08-22 22:34:02 157.39
sell 2023-08-24 00:00:00 156.3 157.39
buy 2023-08-29 09:20:38 158.3
sell 2023-08-30 00:00:00 158.52 158.3
buy 2023-09-21 08:34:45 163.48
2023-09-21 19:44:22 2023-09-22 00:00:00
2023-09-21 19:44:22 2023-09-25 00:00:00
buy_sell 2023-09-26 00:00:00 166.06 163.48
buy 2023-09-21 19:44:22 163.48
2023-09-21 21:29:13 2023-09-22 00:00:00
2023-09-21 21:29:13 2023-09-25 00:00:00
buy_sell 2023-09-26 00:00:00 166.06 163.48
buy 2023-09-21 21:29:13 163.48
sell 2023-10-05 00:00:00 162.2 163.48
buy 2023-10-09 07:19:34 164.99
sell 2023-10-27 00:00:00 142.85 164.99
buy 2023-10-30 01:03:54 144.57
2023-10-30 19:03:37 2023-10-31 00:00:00
buy_sell 2023-11-01 00:00:00 142.49 144.57
buy 2023-11-06 01:39:36 145.48
sell 2023-11-28 00:00:00 145.51 145.48
profit: -20.789999999999935 invest: 1569.93 p

In [73]:
df["profit"].sum()/df["invest"].sum()

-0.01324262865223286

# chromadb (not use)

In [2]:
#https://docs.trychroma.com/usage-guide
%pip install chromadb
%pip install openai
import chromadb
from chromadb.config import Settings



   ---------------------------------------- 0.0/221.4 kB ? eta -:--:--
   -------------------- ------------------- 112.6/221.4 kB 3.2 MB/s eta 0:00:01
   ----------------------------------- ---- 194.6/221.4 kB 2.0 MB/s eta 0:00:01
   -------------------------------------- - 215.0/221.4 kB 1.9 MB/s eta 0:00:01
   -------------------------------------- - 215.0/221.4 kB 1.9 MB/s eta 0:00:01
   -------------------------------------- - 215.0/221.4 kB 1.9 MB/s eta 0:00:01
   -------------------------------------- - 215.0/221.4 kB 1.9 MB/s eta 0:00:01
   -------------------------------------- - 215.0/221.4 kB 1.9 MB/s eta 0:00:01
   -------------------------------------- - 215.0/221.4 kB 1.9 MB/s eta 0:00:01
   -------------------------------------- - 215.0/221.4 kB 1.9 MB/s eta 0:00:01
   -------------------------------------- - 215.0/221.4 kB 1.9 MB/s eta 0:00:01
   -------------------------------------- - 215.0/221.4 kB 1.9 MB/s eta 0:00:01
   -------------------------------------- - 215

In [3]:
persist_directory = '../chroma'
client = chromadb.PersistentClient(path=persist_directory)

In [7]:
%pip install langchain

     ---------------------------------------- 0.0/61.2 kB ? eta -:--:--
     ------------------- ------------------ 30.7/61.2 kB 660.6 kB/s eta 0:00:01
     ------------------- ------------------ 30.7/61.2 kB 660.6 kB/s eta 0:00:01
     ------------------------------- ------ 51.2/61.2 kB 375.8 kB/s eta 0:00:01
     ------------------------------- ------ 51.2/61.2 kB 375.8 kB/s eta 0:00:01
     -------------------------------------- 61.2/61.2 kB 250.8 kB/s eta 0:00:00
   ---------------------------------------- 0.0/809.1 kB ? eta -:--:--
   - ------------------------------------- 30.7/809.1 kB 660.6 kB/s eta 0:00:02
   - ------------------------------------- 41.0/809.1 kB 495.5 kB/s eta 0:00:02
   --- ----------------------------------- 71.7/809.1 kB 563.7 kB/s eta 0:00:02
   ----- -------------------------------- 112.6/809.1 kB 547.6 kB/s eta 0:00:02
   ------ ------------------------------- 143.4/809.1 kB 568.9 kB/s eta 0:00:02
   --------- ---------------------------- 194.6/809.1 kB 

In [38]:
# embedding_function comaperison https://note.com/alexweberk/n/ncccfdab3f4bb
from chromadb.utils import embedding_functions
default_ef = embedding_functions.DefaultEmbeddingFunction()
collection = client.create_collection(name="test",embedding_function=default_ef)


In [37]:
client.delete_collection(name="test")

In [ ]:
collection.add(
    documents=["lorem ipsum...", "doc2", "doc3", ...],
    metadatas=[{"chapter": "3", "verse": "16"}, {"chapter": "3", "verse": "5"}, {"chapter": "29", "verse": "11"}, ...],
    ids=["id1", "id2", "id3", ...]
)

In [43]:
all_data["stock_news"].head()

,symbol,publishedDate,title,image,site,text,url
0,HUBB,2023-12-06 19:32:39,Hubbell (HUBB) Rises As Market Takes a Dip: Ke...,https://cdn.snapi.dev/images/v1/e/a/default7-2...,Zacks Investment Research,Hubbell (HUBB) closed the most recent trading ...,https://www.zacks.com/stock/news/2194483/hubbe...
1,HUBB,2023-12-01 10:31:48,Is Trending Stock Hubbell Inc (HUBB) a Buy Now?,https://cdn.snapi.dev/images/v1/d/t/default44-...,Zacks Investment Research,Zacks.com users have recently been watching Hu...,https://www.zacks.com/stock/news/2192128/is-tr...
2,HUBB,2023-11-26 05:45:00,3 Rock-Solid Dividend Stocks That Just Raised ...,https://cdn.snapi.dev/images/v1/1/n/urlhttps3a...,The Motley Fool,Microsoft is a victim of its own success. Emer...,https://www.fool.com/investing/2023/11/26/3-ro...
3,HUBB,2023-11-20 10:31:45,Hubbell Inc (HUBB) is Attracting Investor Atte...,https://cdn.snapi.dev/images/v1/8/x/default23-...,Zacks Investment Research,Hubbell (HUBB) has received quite a bit of att...,https://www.zacks.com/stock/news/2186541/hubbe...
4,HUBB,2023-11-13 13:27:51,Hubbell: Electrified Operations,https://cdn.snapi.dev/images/v1/4/n/computer-e...,Seeking Alpha,Hubbell has experienced continued growth and m...,https://seekingalpha.com/article/4651258-hubbe...


In [55]:
tmp.loc[1]

symbol                                                        HUBB
publishedDate                                  2023-12-01 10:31:48
title              Is Trending Stock Hubbell Inc (HUBB) a Buy Now?
image            https://cdn.snapi.dev/images/v1/d/t/default44-...
site                                     Zacks Investment Research
text             Zacks.com users have recently been watching Hu...
url              https://www.zacks.com/stock/news/2192128/is-tr...
Name: 1, dtype: object

In [56]:
def embed_text(kind,collection):
    data=all_data[kind]
    if kind=="stock_news":
        embedding_template="symbol:{}\npublishedDate:{}\ntitle:{}\ntext:{}"
        documents=[None]*len(data)
        metadatas=[None]*len(data)
        ids=[None]*len(data)
        print("preprocessing")
        strat=time.time()
        for i in range(len(data)):
            row=data.loc[i]
            documents[i]=embedding_template.format(row["symbol"],row["publishedDate"],row["title"],row["text"])
            metadatas[i]={"id":row.name,"symbol":row["symbol"],"publishedDate":row["publishedDate"],"title":row["title"],"text":row["text"],"site":row["site"]}
            ids[i]=str(row.name)
        print("finished preprocessing in:",time.time()-strat)
        print("embedding")
        strat=time.time()
        collection.add(documents=documents,metadatas=metadatas,ids=ids)
        print("finished embedding in:",time.time()-strat)
    return collection

In [58]:
kind="stock_news"
data=all_data[kind]
if kind=="stock_news":
    embedding_template="symbol:{}\npublishedDate:{}\ntitle:{}\ntext:{}"
    documents=[None]*len(data)
    metadatas=[None]*len(data)
    ids=[None]*len(data)
    print("preprocessing")
    strat=time.time()
    for i in range(len(data)):
        row=data.loc[i]
        documents[i]=embedding_template.format(row["symbol"],row["publishedDate"],row["title"],row["text"])
        metadatas[i]={"id":row.name,"symbol":row["symbol"],"publishedDate":row["publishedDate"],"title":row["title"],"text":row["text"],"site":row["site"]}
        ids[i]=str(row.name)
    print("finished preprocessing in:",time.time()-strat)
    print("embedding")
    strat=time.time()
    collection.add(documents=documents,metadatas=metadatas,ids=ids)
    print("finished embedding in:",time.time()-strat)

preprocessing
finished preprocessing in: 126.46432113647461
embedding


C:\Users\n1201023.STCN2\.cache\chroma\onnx_models\all-MiniLM-L6-v2\onnx.tar.gz: 100%|██████████| 79.3M/79.3M [01:01<00:00, 1.36MiB/s]


KeyboardInterrupt: 

In [ ]:
import tiktoken
# https://note.com/npaka/n/ncb4864df31c9
enc = tiktoken.get_encoding("cl100k_base")
# ex)embedding_template="symbol:{}\npublishedDate:{}\ntitle:{}\ntext:{}",format_columns=["symbol","publishedDate","title","text"]
def count_total_tokens(kind,enc,embedding_template,format_columns):
    data=all_data[kind]
    total=0
    for i in range(len(data)):
        row=data.loc[i]
        text=embedding_template.format((row[col] for col in format_columns))
        tokens = enc.encode(text)
        total+=len(tokens)
    print("tokens:",total,i,"/",len(data))
    return total